# DCGAN 训练：AnimeFace 数据集 (64x64 分辨率)

本项目使用 DCGAN 模型在 anime-face-character-dataset 上训练生成动漫脸图像。
注意：目前模型架构和训练设置过于简单，需要优化。

In [ ]:
# ✅ 安装依赖
!pip install torchvision tqdm
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# ✅ 解压 AnimeFace 数据集
!unzip -q animefaces.zip -d ./data/anime_faces

In [ ]:
# ✅ 定义 DCGAN 模型
class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super().__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 256, 4, 1, 0),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, 2, 1),
            nn.Tanh(),
        )
    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, 2, 1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, 2, 1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 1, 4, 1, 0),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x).view(-1)

In [ ]:
# ✅ 准备训练环境
device = 'cuda' if torch.cuda.is_available() else 'cpu'
z_dim = 100
lr = 0.0002
batch_size = 64
epochs = 20

transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

dataset = torchvision.datasets.ImageFolder('./data/anime_faces', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

G = Generator(z_dim).to(device)
D = Discriminator().to(device)

opt_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
opt_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))
loss_fn = nn.BCELoss()
os.makedirs("output_images", exist_ok=True)

In [ ]:
# ✅ 训练 DCGAN
for epoch in range(epochs):
    for real, _ in tqdm(dataloader):
        real = real.to(device)
        b_size = real.size(0)
        noise = torch.randn(b_size, z_dim, 1, 1, device=device)
        fake = G(noise)

        # Train D
        D_real = D(real)
        D_fake = D(fake.detach())
        loss_D = loss_fn(D_real, torch.ones_like(D_real)) + loss_fn(D_fake, torch.zeros_like(D_fake))
        opt_D.zero_grad(); loss_D.backward(); opt_D.step()

        # Train G
        output = D(fake)
        loss_G = loss_fn(output, torch.ones_like(output))
        opt_G.zero_grad(); loss_G.backward(); opt_G.step()

    save_image(fake.data[:64], f"output_images/epoch_{epoch+1}.png", normalize=True)
    print(f"Epoch {epoch+1} completed.")

In [ ]:
# ✅ 保存模型参数
torch.save(G.state_dict(), 'dcgan_generator.pth')
torch.save(D.state_dict(), 'dcgan_discriminator.pth')
print('✅ 模型已保存为 dcgan_generator.pth 和 dcgan_discriminator.pth')

In [ ]:
# ✅ 生成自定义数量动漫脸图像
G.eval()
num_to_generate = 1000
os.makedirs("output_1000", exist_ok=True)
with torch.no_grad():
    for i in range(num_to_generate):
        z = torch.randn(1, z_dim, 1, 1, device=device)
        img = G(z)
        save_image(img, f"output_1000/{i:04d}.png", normalize=True)
print(f"✅ 生成完成，共生成 {num_to_generate} 张图像")